In [2]:
import torch
import utilities
import numpy as np
import pandas as pd
from sc_sharp import scSHARP
import matplotlib.pyplot as plt

In [3]:
x = np.array([[1,1], [2,2], [3,3], [10,10], [11,11], [12,12]])
preds = [0, -1, 0, 1, -1, 1]

In [4]:
utilities.knn_consensus(x, preds, 2, one_epoch=True)

array([0, 0, 0, 1, 1, 1])

In [5]:
utilities.knn_consensus_batch(x, preds, 2, batch_size = 3, converge=True)

1


/home/users/lewinsda/scSHARP/utilities.py:423: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  dataset  = torch.utils.data.TensorDataset(torch.tensor(counts), torch.tensor(preds))


array([0., 0., 0., 1., 1., 1.], dtype=float32)

In [17]:
#data_path = "/home/groups/ConradLab/daniel/sharp_data/sharp_sims_test/splat_0.7_de_rq/"
data_path = "/home/groups/ConradLab/daniel/sharp_data/sharp_sims_test/splat_0.8_de_rq/"
#data_path = "/home/groups/ConradLab/daniel/sharp_data/jung/"

In [18]:
counts = pd.read_csv(data_path + "query_counts.csv", index_col = 0)
#counts = pd.read_csv(data_path + "counts.csv", index_col = 0)
counts.head()

,Gene1,Gene2,Gene3,Gene4,Gene5,Gene6,Gene7,Gene8,Gene9,Gene10,...,Gene33685,Gene33686,Gene33687,Gene33688,Gene33689,Gene33690,Gene33691,Gene33692,Gene33693,Gene33694
Cell1001,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Cell1002,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Cell1003,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Cell1004,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Cell1005,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
all_labels = pd.read_csv(data_path + "preds.csv", index_col=0)
all_labels.head()

,scina,scsorter,sctype,singler,scpred
Cell1001,NaN,Group4,Group4,Group4,Group4
Cell1002,Group3,Group3,Group3,Group3,Group3
Cell1003,Group3,Group3,Group3,Group3,Group3
Cell1004,NaN,Group3,Group2,Group3,Group3
Cell1005,NaN,Group3,Group3,Group3,Group3


In [20]:
X, keep_cells,keep_genes,pca_obj = utilities.preprocess(np.array(counts), scale=False, run_pca=True)
X.shape

(998, 500)

In [21]:
_,marker_names = utilities.read_marker_file(data_path + "markers.txt")

In [22]:
all_labels_factored = utilities.factorize_df(all_labels, marker_names)
encoded_labels = utilities.encode_predictions(all_labels_factored)
confident_labels = utilities.get_consensus_labels(encoded_labels, necessary_vote = .51)
train_nodes = np.where(confident_labels != -1)[0]
test_nodes = np.where(confident_labels == -1)[0]

In [23]:
real_y = pd.read_csv(data_path + "query_meta.csv", index_col= 0).Group
#real_y = pd.read_csv(data_path + "labels.csv", index_col= 0).iloc[:,0]
real_y = pd.factorize(real_y, sort=True)[0]
real_y = real_y[keep_cells]

In [16]:
utilities.validation_metrics(torch.tensor(real_y), torch.tensor(confident_labels), train_nodes, test_nodes)

(0.8078034520149231,
 array([[  0,   0,   0,   0],
        [ 32, 136,  74,   2],
        [  4,  10, 142,   0],
        [ 10,   1,   0, 281]]),
 0.8653250932693481,
 array([[136,  74,   2],
        [ 10, 142,   0],
        [  1,   0, 281]]),
 0.0,
 array([[ 0,  0,  0,  0],
        [32,  0,  0,  0],
        [ 4,  0,  0,  0],
        [10,  0,  0,  0]]))

In [15]:
# one epoch
for i in [10,50]:
    preds = utilities.knn_consensus_batch(X, confident_labels, i, converge=False, one_epoch=True, batch_size=1000)
    print(i)
    print(utilities.validation_metrics(torch.tensor(real_y), torch.tensor(preds), train_nodes,test_nodes))
# one epoch
for i in [10,50]:
    preds = utilities.knn_consensus_batch(X, confident_labels, i, converge=False, one_epoch=True, batch_size=1000, keep_conf=True)
    print(i)
    print(utilities.validation_metrics(torch.tensor(real_y), torch.tensor(preds), train_nodes, test_nodes))

10
(0.8684971332550049, array([[  0,   0,   0,   0],
       [  7, 159,  77,   1],
       [  1,   5, 150,   0],
       [  0,   0,   0, 292]]), 0.8792569637298584, array([[139,  72,   1],
       [  5, 147,   0],
       [  0,   0, 282]]), 0.717391312122345, array([[ 0,  0,  0,  0],
       [ 7, 20,  5,  0],
       [ 1,  0,  3,  0],
       [ 0,  0,  0, 10]]))
50
(0.8728323578834534, array([[163,  73,   8],
       [  5, 149,   2],
       [  0,   0, 292]]), 0.8746129870414734, array([[138,  69,   5],
       [  5, 145,   2],
       [  0,   0, 282]]), 0.8478260636329651, array([[25,  4,  3],
       [ 0,  4,  0],
       [ 0,  0, 10]]))
10
(0.8554913401603699, array([[  0,   0,   0,   0],
       [  7, 156,  79,   2],
       [  1,  10, 145,   0],
       [  0,   1,   0, 291]]), 0.8653250932693481, array([[136,  74,   2],
       [ 10, 142,   0],
       [  1,   0, 281]]), 0.717391312122345, array([[ 0,  0,  0,  0],
       [ 7, 20,  5,  0],
       [ 1,  0,  3,  0],
       [ 0,  0,  0, 10]]))
50
(0.864

In [9]:
# one epoch
for i in [10,50,100, 200, 400, 500]:
    preds = utilities.knn_consensus(X, confident_labels, i, converge=False, one_epoch=True)
    print(i)
    print(utilities.validation_metrics(torch.tensor(real_y), torch.tensor(preds), range(len(real_y)), range(len(real_y)))[0:2])


10
(0.37374749779701233, array([[  0,   0,   0,   0,   0],
       [  0,  69,   0,   0, 185],
       [  1,   2,   1,   1, 239],
       [  1,   0,   0,  58, 196],
       [  0,   0,   0,   0, 245]]))
50
(0.2635270655155182, array([[  7,   0,   0, 247],
       [  0,   0,   0, 244],
       [  0,   0,  11, 244],
       [  0,   0,   0, 245]]))
100
(0.24949899315834045, array([[  0,   0,   0, 254],
       [  0,   0,   0, 244],
       [  0,   0,   4, 251],
       [  0,   0,   0, 245]]))
200
(0.24849699437618256, array([[  1,   0,   0, 253],
       [  0,   0,   0, 244],
       [  0,   0,   2, 253],
       [  0,   0,   0, 245]]))
400
(0.26853707432746887, array([[ 15,   0,   0, 239],
       [  0,   0,   0, 244],
       [  0,   0,   8, 247],
       [  0,   0,   0, 245]]))
500
(0.29859718680381775, array([[ 42,   0,   0, 212],
       [  0,   0,   0, 244],
       [  0,   0,  11, 244],
       [  0,   0,   0, 245]]))


In [10]:
# one epoch
for i in [10,50,100, 200, 400, 500]:
    preds = utilities.knn_consensus_batch(X, confident_labels, i, converge=False, one_epoch=True, batch_size=1000)
    print(i)
    print(utilities.validation_metrics(torch.tensor(real_y), torch.tensor(preds), range(len(real_y)), range(len(real_y)))[0:2])

10
(0.37374749779701233, array([[  0,   0,   0,   0,   0],
       [  0,  69,   0,   0, 185],
       [  1,   2,   1,   1, 239],
       [  1,   0,   0,  58, 196],
       [  0,   0,   0,   0, 245]]))
50
(0.2635270655155182, array([[  7,   0,   0, 247],
       [  0,   0,   0, 244],
       [  0,   0,  11, 244],
       [  0,   0,   0, 245]]))
100
(0.24949899315834045, array([[  0,   0,   0, 254],
       [  0,   0,   0, 244],
       [  0,   0,   4, 251],
       [  0,   0,   0, 245]]))
200
(0.24849699437618256, array([[  1,   0,   0, 253],
       [  0,   0,   0, 244],
       [  0,   0,   2, 253],
       [  0,   0,   0, 245]]))
400
(0.26853707432746887, array([[ 15,   0,   0, 239],
       [  0,   0,   0, 244],
       [  0,   0,   8, 247],
       [  0,   0,   0, 245]]))
500
(0.29859718680381775, array([[ 42,   0,   0, 212],
       [  0,   0,   0, 244],
       [  0,   0,  11, 244],
       [  0,   0,   0, 245]]))


In [27]:
# until all labelled
for i in [10,50,100, 200, 400, 500]:
    preds = utilities.knn_consensus(X, confident_labels, i, converge=False, one_epoch=False)
    print(i)
    print(utilities.validation_metrics(torch.tensor(real_y), torch.tensor(preds), range(len(real_y)), range(len(real_y)))[0:2])

1
2
10
(0.24549098312854767, array([[  0,   0,   0, 254],
       [  0,   0,   0, 244],
       [  0,   0,   0, 255],
       [  0,   0,   0, 245]]))
50
(0.2635270655155182, array([[  7,   0,   0, 247],
       [  0,   0,   0, 244],
       [  0,   0,  11, 244],
       [  0,   0,   0, 245]]))
100
(0.24949899315834045, array([[  0,   0,   0, 254],
       [  0,   0,   0, 244],
       [  0,   0,   4, 251],
       [  0,   0,   0, 245]]))
200
(0.24849699437618256, array([[  1,   0,   0, 253],
       [  0,   0,   0, 244],
       [  0,   0,   2, 253],
       [  0,   0,   0, 245]]))
400
(0.26853707432746887, array([[ 15,   0,   0, 239],
       [  0,   0,   0, 244],
       [  0,   0,   8, 247],
       [  0,   0,   0, 245]]))
500
(0.29859718680381775, array([[ 42,   0,   0, 212],
       [  0,   0,   0, 244],
       [  0,   0,  11, 244],
       [  0,   0,   0, 245]]))


In [11]:
# until all labelled
for i in [10,50,100, 200, 400, 500]:
    preds = utilities.knn_consensus_batch(X, confident_labels, i, converge=False, one_epoch=False, batch_size=1000)
    print(i)
    print(utilities.validation_metrics(torch.tensor(real_y), torch.tensor(preds), range(len(real_y)), range(len(real_y)))[0:2])

1


/home/users/lewinsda/scSHARP/utilities.py:423: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  dataset  = torch.utils.data.TensorDataset(torch.tensor(counts), torch.tensor(preds))


2
10
(0.24549098312854767, array([[  0,   0,   0, 254],
       [  0,   0,   0, 244],
       [  0,   0,   0, 255],
       [  0,   0,   0, 245]]))
50
(0.2635270655155182, array([[  7,   0,   0, 247],
       [  0,   0,   0, 244],
       [  0,   0,  11, 244],
       [  0,   0,   0, 245]]))
100
(0.24949899315834045, array([[  0,   0,   0, 254],
       [  0,   0,   0, 244],
       [  0,   0,   4, 251],
       [  0,   0,   0, 245]]))
200
(0.24849699437618256, array([[  1,   0,   0, 253],
       [  0,   0,   0, 244],
       [  0,   0,   2, 253],
       [  0,   0,   0, 245]]))
400
(0.26853707432746887, array([[ 15,   0,   0, 239],
       [  0,   0,   0, 244],
       [  0,   0,   8, 247],
       [  0,   0,   0, 245]]))
500
(0.29859718680381775, array([[ 42,   0,   0, 212],
       [  0,   0,   0, 244],
       [  0,   0,  11, 244],
       [  0,   0,   0, 245]]))


In [28]:
# until converge
for i in [10,50,100, 200, 400, 500]:
    preds = utilities.knn_consensus(X, confident_labels, i, converge=True, one_epoch=False)
    print(i)
    print(utilities.validation_metrics(torch.tensor(real_y), torch.tensor(preds), range(len(real_y)), range(len(real_y)))[0:2])

1
2
10
(0.24549098312854767, array([[  0,   0,   0, 254],
       [  0,   0,   0, 244],
       [  0,   0,   0, 255],
       [  0,   0,   0, 245]]))
1
50
(0.24549098312854767, array([[  0,   0,   0, 254],
       [  0,   0,   0, 244],
       [  0,   0,   0, 255],
       [  0,   0,   0, 245]]))
1
100
(0.24549098312854767, array([[  0,   0,   0, 254],
       [  0,   0,   0, 244],
       [  0,   0,   0, 255],
       [  0,   0,   0, 245]]))
1
200
(0.24549098312854767, array([[  0,   0,   0, 254],
       [  0,   0,   0, 244],
       [  0,   0,   0, 255],
       [  0,   0,   0, 245]]))
1
400
(0.24549098312854767, array([[  0,   0,   0, 254],
       [  0,   0,   0, 244],
       [  0,   0,   0, 255],
       [  0,   0,   0, 245]]))
1
2
500
(0.24549098312854767, array([[  0,   0,   0, 254],
       [  0,   0,   0, 244],
       [  0,   0,   0, 255],
       [  0,   0,   0, 245]]))


In [16]:
# until converge batch
for i in [5,10,50,100]:
    preds = utilities.knn_consensus_batch(X, confident_labels, i, batch_size = 300, converge=True, one_epoch=False)
    print(i)
    print(utilities.validation_metrics(torch.tensor(real_y), torch.tensor(preds), range(len(real_y)), range(len(real_y)))[0:2])

1


/home/users/lewinsda/scSHARP/utilities.py:423: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  dataset  = torch.utils.data.TensorDataset(torch.tensor(counts), torch.tensor(preds))


5
(0.8786126971244812, array([[  0,   0,   0,   0],
       [  3, 167,  71,   3],
       [  1,   5, 150,   0],
       [  1,   0,   0, 291]]))
1


/home/users/lewinsda/scSHARP/utilities.py:423: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  dataset  = torch.utils.data.TensorDataset(torch.tensor(counts), torch.tensor(preds))


10
(0.8771676421165466, array([[  0,   0,   0,   0],
       [  4, 163,  75,   2],
       [  0,   4, 152,   0],
       [  0,   0,   0, 292]]))
1


/home/users/lewinsda/scSHARP/utilities.py:423: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  dataset  = torch.utils.data.TensorDataset(torch.tensor(counts), torch.tensor(preds))


50
(0.8352600932121277, array([[150,  92,   2],
       [ 16, 139,   1],
       [  0,   3, 289]]))
1


/home/users/lewinsda/scSHARP/utilities.py:423: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  dataset  = torch.utils.data.TensorDataset(torch.tensor(counts), torch.tensor(preds))


2
100
(0.6994219422340393, array([[ 76, 133,  35],
       [ 17, 119,  20],
       [  2,   1, 289]]))


In [10]:
datasets = ["/home/groups/ConradLab/daniel/sharp_data/sharp_sims_test/splat_0.7_de_rq/","/home/groups/ConradLab/daniel/sharp_data/sharp_sims_test/splat_0.8_de_rq/","/home/groups/ConradLab/daniel/sharp_data/jung/"]

for dataset in datasets:
    if dataset == "/home/groups/ConradLab/daniel/sharp_data/jung/":
        counts = pd.read_csv(dataset + "counts.csv", index_col = 0)
        real_y = pd.read_csv(dataset + "labels.csv", index_col= 0).iloc[:,0]
        data_name = "jung"
    else:
        counts = pd.read_csv(dataset + "query_counts.csv", index_col = 0)
        real_y = pd.read_csv(dataset + "query_meta.csv", index_col= 0).Group
        if dataset.split("/")[7] == "splat_0.7_de_rq":
            data_name = "sim_0.7"
        else: data_name = "sim_0.8"
        
    all_labels = pd.read_csv(dataset + "preds.csv", index_col=0)
    X, keep_cells,keep_genes,pca_obj = utilities.preprocess(np.array(counts), scale=False, run_pca=True)
    _,marker_names = utilities.read_marker_file(dataset + "markers.txt")
    all_labels_factored = utilities.factorize_df(all_labels, marker_names)
    encoded_labels = utilities.encode_predictions(all_labels_factored)
    confident_labels = utilities.get_consensus_labels(encoded_labels, necessary_vote = .51)
    real_y = pd.factorize(real_y, sort=True)[0]
    real_y = real_y[keep_cells]
    
    neighbors = [10,50,100, 200, 300, 400, 500, 600, 700, 800, 900]
    accuracies = []
    # one epoch
    for i in neighbors:
        preds = utilities.knn_consensus_batch(X, confident_labels, i, converge=False, one_epoch=True, batch_size=1000)
        accuracies.append(utilities.validation_metrics(torch.tensor(real_y), torch.tensor(preds), range(len(real_y)), range(len(real_y)))[0])
    
    plt.scatter(neighbors, accuracies)
    plt.xlabel("Neighbors")
    plt.ylabel("Accuracy")
    plt.title("One Epoch")
    plt.savefig('figures/' + data_name + '_knn_plot_one.pdf')
    plt.clf()
    accuracies = []
    
    # until all labelled
    for i in neighbors:
        preds = utilities.knn_consensus_batch(X, confident_labels, i, converge=False, one_epoch=False, batch_size=1000)
        accuracies.append(utilities.validation_metrics(torch.tensor(real_y), torch.tensor(preds), range(len(real_y)), range(len(real_y)))[0])
    plt.scatter(neighbors, accuracies)
    plt.xlabel("Neighbors")
    plt.ylabel("Accuracy")
    plt.title("Iterate until all labeled")
    plt.savefig('figures/' + data_name + '_knn_plot_all.pdf')
    plt.clf()
    accuracies = []
    
    # until converge
    for i in neighbors:
        preds = utilities.knn_consensus_batch(X, confident_labels, i, converge=True, one_epoch=False, batch_size=1000)
        accuracies.append(utilities.validation_metrics(torch.tensor(real_y), torch.tensor(preds), range(len(real_y)), range(len(real_y)))[0])
    plt.scatter(neighbors, accuracies)
    plt.xlabel("Neighbors")
    plt.ylabel("Accuracy")
    plt.title("Iterate until convergence")
    plt.savefig('figures/' + data_name + '_knn_plot_converge.pdf')
    plt.clf()
        

1


/home/users/lewinsda/scSHARP/utilities.py:423: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  dataset  = torch.utils.data.TensorDataset(torch.tensor(counts), torch.tensor(preds))


1


/home/users/lewinsda/scSHARP/utilities.py:423: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  dataset  = torch.utils.data.TensorDataset(torch.tensor(counts), torch.tensor(preds))


1


...

1


/home/users/lewinsda/scSHARP/utilities.py:423: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  dataset  = torch.utils.data.TensorDataset(torch.tensor(counts), torch.tensor(preds))


1


/home/users/lewinsda/scSHARP/utilities.py:423: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  dataset  = torch.utils.data.TensorDataset(torch.tensor(counts), torch.tensor(preds))


<Figure size 432x288 with 0 Axes>

In [ ]:
preds = utilities.knn_consensus(X, confident_labels, 700, converge=True, one_epoch=False)

print(utilities.validation_metrics(torch.tensor(real_y), torch.tensor(preds), range(len(real_y)), range(len(real_y)))[0:2])

In [ ]:
preds.shape

In [76]:
print(utilities.validation_metrics(torch.tensor(real_y), torch.tensor(confident_labels), range(len(real_y)), range(len(real_y)))[0:2])

(0.8078034520149231, array([[  0,   0,   0,   0],
       [ 32, 136,  74,   2],
       [  4,  10, 142,   0],
       [ 10,   1,   0, 281]]))


In [14]:
#0.7 test
preds = utilities.knn_consensus_batch(X, confident_labels, 300, converge=False, one_epoch=True, batch_size=1000)
print(utilities.validation_metrics(torch.tensor(real_y), torch.tensor(preds), train_nodes, test_nodes))

(0.3056112229824066, array([[ 55,   0,   0, 199],
       [  0,   0,   0, 244],
       [  0,   0,   5, 250],
       [  0,   0,   0, 245]]), 0.33135393261909485, array([[ 55,   0,   0, 195],
       [  0,   0,   0, 127],
       [  0,   0,   5, 241],
       [  0,   0,   0, 219]]), 0.1666666716337204, array([[  0,   0,   0,   4],
       [  0,   0,   0, 117],
       [  0,   0,   0,   9],
       [  0,   0,   0,  26]]))


In [22]:
#0.8 test
preds = utilities.knn_consensus_batch(X, confident_labels, 10, converge=False, one_epoch=True, batch_size=1000)
print(utilities.validation_metrics(torch.tensor(real_y), torch.tensor(preds), train_nodes, test_nodes))

(0.37374749779701233, array([[  0,   0,   0,   0,   0],
       [  0,  69,   0,   0, 185],
       [  1,   2,   1,   1, 239],
       [  1,   0,   0,  58, 196],
       [  0,   0,   0,   0, 245]]), 0.394304484128952, array([[ 69,   0,   0, 183],
       [  2,   1,   1, 180],
       [  0,   0,  57, 187],
       [  0,   0,   0, 233]]), 0.15294118225574493, array([[ 0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  2],
       [ 1,  0,  0,  0, 59],
       [ 1,  0,  0,  1,  9],
       [ 0,  0,  0,  0, 12]]))


In [30]:
#jung test
preds = utilities.knn_consensus_batch(X, confident_labels, 200, converge=True, one_epoch=False, batch_size=1000)
print(utilities.validation_metrics(torch.tensor(real_y), torch.tensor(preds), train_nodes, test_nodes))

1


/home/users/lewinsda/scSHARP/utilities.py:423: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  dataset  = torch.utils.data.TensorDataset(torch.tensor(counts), torch.tensor(preds))


(0.8742774724960327, array([[167,  72,   5],
       [  9, 146,   1],
       [  0,   0, 292]]), 0.8761609792709351, array([[141,  68,   3],
       [  8, 143,   1],
       [  0,   0, 282]]), 0.8478260636329651, array([[26,  4,  2],
       [ 1,  3,  0],
       [ 0,  0, 10]]))


In [26]:
#0.8 test keep conf
preds = utilities.knn_consensus_batch(X, confident_labels, 10, converge=True, one_epoch=False, batch_size=1000, keep_conf=True)
print(utilities.validation_metrics(torch.tensor(real_y), torch.tensor(preds), train_nodes, test_nodes))

1


/home/users/lewinsda/scSHARP/utilities.py:423: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  dataset  = torch.utils.data.TensorDataset(torch.tensor(counts), torch.tensor(preds))


(0.9028056263923645, array([[  0,   0,   0,   0,   0],
       [  0, 251,   1,   0,   2],
       [  1,  14, 167,   0,  62],
       [  1,   1,   0, 244,   9],
       [  0,   1,   3,   2, 239]]), 0.9726177453994751, array([[251,   1,   0,   0],
       [ 14, 167,   0,   3],
       [  1,   0, 243,   0],
       [  1,   3,   2, 227]]), 0.15294118225574493, array([[ 0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  2],
       [ 1,  0,  0,  0, 59],
       [ 1,  0,  0,  1,  9],
       [ 0,  0,  0,  0, 12]]))
